In [8]:

# coding: utf-8

# In[ ]:


from __future__ import print_function

import sys, os
import numpy as np
from time import time
from get_vector import get_intt
import matplotlib.pyplot as plt 

sys.path.append('/home/xilinx')
os.environ['XILINX_XRT'] = '/usr'
from pynq import Overlay
from pynq import allocate

if __name__ == "__main__":
    print("Entry:", sys.argv[0])
    print("System argument(s):", len(sys.argv))
    print("Start of \"" + sys.argv[0] + "\"")
    
    # Overlay
    ol = Overlay("intt.bit")
    ipiNTT = ol.iNTT_0


    # Memory allocate
    inBuffer0 = allocate(shape=(1024,), dtype=np.int16)
    test = [0 for i in range(1024)]
    
    sum = 0
    for j in range(10):
        iNTT_in = open("mq_intt_in.txt", "r+")
        iNTT_in.seek(0)
        for i in range(1024):
            line = iNTT_in.readline()
            test[i] = int(line)
        iNTT_in.close()
    
        temp = get_intt(test)
        for i in range(1024):
            inBuffer0[i] = int(temp[i])
    
        timeKernelStart = time()
    # ============= Set Kernel Argument ==============
    # 0x00 : Control signals
    #        bit 0  - ap_start (Read/Write/COH)
    #        bit 1  - ap_done (Read/COR)
    #        bit 2  - ap_idle (Read)
    #        bit 3  - ap_ready (Read/COR)
    #        bit 7  - auto_restart (Read/Write)
    #        bit 9  - interrupt (Read)
    #        others - reserved
    # 0x04 : Global Interrupt Enable Register
    #        bit 0  - Global Interrupt Enable (Read/Write)
    #        others - reserved
    # 0x08 : IP Interrupt Enable Register (Read/Write)
    #        bit 0 - enable ap_done interrupt (Read/Write)
    #        bit 1 - enable ap_ready interrupt (Read/Write)
    #        others - reserved
    # 0x0c : IP Interrupt Status Register (Read/COR)
    #        bit 0 - ap_done (Read/COR)
    #        bit 1 - ap_ready (Read/COR)
    #        others - reserved
    # 0x10 : Data signal of a
    #        bit 31~0 - a[31:0] (Read/Write)
    # 0x14 : Data signal of a
    #        bit 31~0 - a[63:32] (Read/Write)
    # 0x18 : reserved
    # 0x1c : Data signal of logn
    #        bit 31~0 - logn[31:0] (Read/Write)
    
        logn = 10
        ipiNTT.write(0x1C, logn) # Coefficient

    # ================ Data Transfer =================
        ipiNTT.write(0x10, inBuffer0.device_address) # 0x10 bit 31:0
        ipiNTT.write(0x14, inBuffer0.device_address) # 0x14 bit 63:32

    # =================== Start ======================
        ipiNTT.write(0x00, 0x01) # 0x00: bit 0: 0->1: start
        while (ipiNTT.read(0x00) & 0x4) == 0x0:
            continue
        timeKernelEnd = time()
        print("Kernel execution time: " + str(timeKernelEnd - timeKernelStart) + " s")
        sum += timeKernelEnd - timeKernelStart
    
    # ================== Compare =====================
        error_cnt = 0
        iNTT_out = open("mq_intt_out.txt", "r+")
        iNTT_out.seek(0)
        for i in range(1024):
            line = iNTT_out.readline()
            golden = int(line)
            if inBuffer0[i] != golden:
                print(f"Failed, expect value: {golden:d}, got value: {inBuffer0[i]:d}")
                error_cnt += 1
            #else:
            #    print(f"Calculate value: {inBuffer0[i]:d}")
        iNTT_out.close()
        if error_cnt == 0:
            print("Test Passed!!")
        else:
            print("Test Failed!!")

    print("Average time:", sum/10)
    print("============================")
    print("Error count : ", error_cnt)
    print("Exit process")

Entry: /usr/local/share/pynq-venv/lib/python3.8/site-packages/ipykernel_launcher.py
System argument(s): 3
Start of "/usr/local/share/pynq-venv/lib/python3.8/site-packages/ipykernel_launcher.py"
Kernel execution time: 0.000186920166015625 s
Test Passed!!
Kernel execution time: 0.0001728534698486328 s
Test Passed!!
Kernel execution time: 0.00018262863159179688 s
Test Passed!!
Kernel execution time: 0.0001728534698486328 s
Test Passed!!
Kernel execution time: 0.00017404556274414062 s
Test Passed!!
Kernel execution time: 0.00017213821411132812 s
Test Passed!!
Kernel execution time: 0.0001671314239501953 s
Test Passed!!
Kernel execution time: 0.00017571449279785156 s
Test Passed!!
Kernel execution time: 0.0001857280731201172 s
Test Passed!!
Kernel execution time: 0.00018072128295898438 s
Test Passed!!
Average time: 0.00017707347869873048
Error count :  0
Exit process
